#**Final Project: Predicting Whether a Person's Eyes are Open or Closed**

# *Names: (Aleks Lazowski, Chris Chang, Jacinto Lemarroy, Shrey Sood)*

#*Background:*

According to the National Highway Traffic Safety Administration, there are approximately 100,000 car crashes, 800 fatalities, and 50,000 injuries every year related to drowsy-driving in the US (nsc, n.d.). To help combat this issue, our team decided to come up with image classification algorithms trained on 4000 images with two classes, open and closed, to detect whether a person's eyes are open or not. We hope this would be the first step to more improved algorithms to come in reducing the number of drowsy-driving fatalities not only in the US but worldwide as well.

## Importing libraries and modules

In [ ]:
#import libraries
from tensorflow import keras
import tensorflow as tf
from keras.layers import Dense,Dropout,Activation,Add,MaxPooling2D,Conv2D,Flatten
from keras.models import Sequential 
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
#from keras.applications import VGG19
from keras import layers
from tensorflow.keras.layers import Input
from keras.preprocessing import image
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from keras.applications.vgg19 import VGG19
model = VGG19(weights='imagenet')
from tensorflow.keras.applications import ResNet50
import os

# import the necessary packages - MobileNet

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import cv2

## Data import from directory and train, test, validation split

Our original dataset contained over 40,000 images of open and closed eyes.  We wanted to take a portion of that data to make it easier for our models to run and data to import.  We settled with 4,000 images, evenly split into open and closed eyes.  These 4,000 were randomly selected as well.  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
basePath = "/content/drive/MyDrive/BA865/Final Project/train-2"

In [ ]:
pip install split-folders

In [ ]:
import splitfolders
splitfolders.ratio(basePath, output="output", seed=1337, ratio=(.8, 0.1,0.1)) 

ValueError: ignored

Using splitfolders we organized the data into training, test, and validation splits on the file system. The labels are inferred from the folder names. 

In [ ]:
#import train and test data
from tensorflow.keras.utils import image_dataset_from_directory

base_dir = "/content/output"
  
train_dataset = image_dataset_from_directory(
    base_dir + "/train/",
    image_size=(224, 224),
    batch_size=32, shuffle=True)
validation_dataset = image_dataset_from_directory(
    base_dir + "/val/",
    image_size=(224, 224),
    batch_size=32, shuffle=True)
test_dataset = image_dataset_from_directory(
    base_dir + "/test/",
    image_size=(224, 224),
    batch_size=32, shuffle=True)

In [ ]:
for data_batch, labels_batch in train_dataset:
     print("data batch shape:", data_batch.shape)
     print("labels batch shape:", labels_batch.shape)
     break

## Exploratory Data Analysis

Here we delve into what our data looks like. We look at examples of the images as well as the class types. We visually represent the distribution of the classes as well as the data split into train, test and validation. As you can appreciate, our data is evenly distributed between the two classes (open and closed eyes) and amongst the data splits.

In [ ]:
# What our dataset looks like
plt.figure(figsize=(10, 10))
class_names = train_dataset.class_names
for images, labels in train_dataset.take(1):
    for i in range(32):
        ax = plt.subplot(6, 6, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
print(class_names)

In [ ]:
#Data visualization of class types
fig, ax = plt.subplots()
ax.bar("Data",4000, label="Data",color="y")
ax.bar("Closed_eyes",2000,label="Closed_eyes",color="g")
ax.bar("Open_eyes",2000,label="Open_eyes",color="b")
ax.legend();

In [ ]:
#Data visualization of data split
fig, ax = plt.subplots()
ax.bar("Data",4000,label="Data",color="y")
ax.bar("train",3200,label="Train",color="g")
ax.bar("test",800,label="Test",color="b")
ax.bar("val",800,label="Test",color="b")
ax.legend();

# Defining Our Model: Binary Outcome NN model

First we implemented a dense neural network model with a binary outcome in order to test its accuracy when predicting image classification

In [ ]:
from tensorflow import keras 
from tensorflow.keras import layers
  
inputs = keras.Input(shape=(224, 224, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.GlobalAveragePooling2D()(x) #before it was flatten
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="drive/My Drive/Teaching/Courses/BA 865/BA865-2022/Week 5/convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss")
]

In [ ]:
history = model.fit(
    train_dataset,
    epochs=50, 
    batch_size = 100, # *default is 32
    # INC adjust bs.. one epoch will be less iterations
    # wont learn 100% acc
    validation_data=validation_dataset,
    callbacks=callbacks)

Let's plot loss over training iterations

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['val_accuracy'],c="b")
plt.plot(history.history['accuracy'],c="r")

plt.legend(['Validation Acc','Training Acc'])
plt.show()

Our model does fairly well in reaching high accuracy of above 90% with a few drops in accuracy but continuing to increase with the epochs.

In [ ]:
test_model = keras.models.load_model("drive/My Drive/Teaching/Courses/BA 865/BA865-2022/Week 5/convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset) 
print(f"Test accuracy: {test_acc}")

# VGG Model

We will use a pre-trained model (VGG19) to analyze the potential impact and improvement on our previous dense model.  VGG is one of the most popular pre-trained models for image classification.  Furthermore, we added Imagenet as a weight.  This VGG model was meant to train a model that can correctly classify an input image into 1,000 separate object categories.  Due to the background of this model, ImageNet challenge is considered the de facto benchmark for computer vision classification algorithms which is why we chose it to test our accuracy and predict open and closed images. 


In [ ]:
vgg_model =  VGG19(include_top=True , weights='imagenet')
for models in vgg_model.layers:
  models.trainable= False
vgg_model = keras.Model(inputs=vgg_model.input, outputs=vgg_model.layers[-2].output)
vgg = keras.Sequential()
for layer in vgg_model.layers:
  vgg.add(layer)
vgg.add(Dropout(0.2))
vgg.add(Dense(2, activation='softmax'))

In [ ]:
vgg.compile(optimizer=keras.optimizers.Adam(0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])


In [ ]:
#fit our model
history = vgg.fit(train_dataset,validation_data=validation_dataset,epochs = 3)

In [ ]:
#evaluate test_data
vgg.evaluate(test_dataset)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['val_accuracy'],c="b")
plt.plot(history.history['accuracy'],c="r")
plt.legend(['Validation Acc','Training Acc'])
plt.show()

# MobileNet

In [ ]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 7
BS = 32

In [ ]:
DIRECTORY = "/content/drive/MyDrive/BA865/Final Project/train-2"
CATEGORIES = ["Closed_Eyes", "Open_Eyes"]

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")

data = []
labels = []

for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)

        data.append(image)
        labels.append(category)

In [ ]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)

In [ ]:
(trainX, validX, trainY, validY) = train_test_split(data, labels,
    test_size=0.20, stratify=labels, random_state=42)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [ ]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

baseModel.summary()

In [ ]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="sigmoid")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers[:65]:
    layer.trainable = False
for layer in baseModel.layers[65:]:
    layer.trainable = True
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])

# train the head of the network
print("[INFO] training head...")
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(validX, validY),
    validation_steps=len(validX) // BS,
    epochs=EPOCHS)

In [ ]:
# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

In [ ]:
# plotting training accuracy & validation accuracy
plt.plot(H.history['val_accuracy'],c="b")
plt.plot(H.history['accuracy'],c="r")
plt.legend(['Validation Acc','Training Acc'])
plt.show()

# Image Prediction

We will upload new images to test whether our model can correctly identify eyes close or open.  We used images from the dataset and from google to test if the model can accurately predict variation in images.

In [ ]:
#import libraries
from tensorflow import keras
import tensorflow as tf
from keras.layers import Dense,Dropout,Activation,Add,MaxPooling2D,Conv2D,Flatten
from keras.models import Sequential 
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
#from keras.applications import VGG19
from keras import layers
from tensorflow.keras.layers import Input
from keras.preprocessing import image
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from keras.applications.vgg19 import VGG19
model = VGG19(weights='imagenet')
from tensorflow.keras.applications import ResNet50
import os

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import cv2

In [ ]:
from tensorflow.keras.layers import Input

In [ ]:
# image 1
image_path = "/content/output/test/Closed_Eyes/s0014_00008_0_0_0_0_0_01.png"
new_img = image.load_img(image_path, target_size=(224, 224))
img = image.img_to_array(new_img)
img = np.expand_dims(img, axis=0)
prediction = vgg.predict(img)
prediction = np.argmax(prediction,axis=1)
print(prediction)
print(class_names[prediction[0]])
plt.imshow(new_img)

In [ ]:
#image 2
image_path = "/content/drive/MyDrive/BA865/Final Project/test/closedeye.jpg"
new_img = image.load_img(image_path, target_size=(224, 224))
img = image.img_to_array(new_img)
img = np.expand_dims(img, axis=0)
prediction = vgg.predict(img)
prediction = np.argmax(prediction,axis=1)
print(prediction)
print(class_names[prediction[0]])
plt.imshow(new_img)

In [ ]:
# image 3
image_path = "/content/output/test/Open_Eyes/s0014_07731_0_0_1_1_1_02.png"
new_img = image.load_img(image_path, target_size=(224, 224))
img = image.img_to_array(new_img)
img = np.expand_dims(img, axis=0)
prediction = vgg.predict(img)
prediction = np.argmax(prediction,axis=1)
print(prediction)
print(class_names[prediction[0]])
plt.imshow(new_img)

In [ ]:
# image 4
image_path = "/content/drive/MyDrive/BA865/Final Project/test/opneye.jpg"
new_img = image.load_img(image_path, target_size=(224, 224))
img = image.img_to_array(new_img)
img = np.expand_dims(img, axis=0)
prediction = vgg.predict(img)
prediction = np.argmax(prediction,axis=1)
print(prediction)
print(class_names[prediction[0]])
plt.imshow(new_img)

In [ ]:
# Using MobileNet
img_size = 224
img_array = cv2.imread('/content/drive/MyDrive/BA865/Final Project/test/closedeye.jpg',cv2.IMREAD_GRAYSCALE)
backtorgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2BGR)
new_array = cv2.resize(backtorgb, (img_size, img_size))
X_input = np.array(new_array).reshape(1, img_size, img_size, 3)
X_input = X_input/255.0
prediction = model.predict(X_input)
prediction

In [ ]:
img = cv2.imread('/content/drive/MyDrive/BA865/Final Project/test/closedeye.jpg')
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))